### Day 23

### Part 1:
- 4 types of amphipods:
    - Amber
    - Bronze
    - Copper
    - Desert
- Burrow:
    - Hallway (start empty)
    - 4 side rooms (starts full)
- Amphipods start in mixed rooms, but we want them to be one type per room
- Different costs to move each one:
    - Amber : 1 per space
    - Bronze: 10 per space
    - Copper: 100 per space
    - Desert: 1000 per space
- Find solution with least total energy
- Constraints:
    - Can't stop in front of a door in the hallway
    - Can't enter a room unless they are going to stay there
    - Can only stop in a hallway once. Next move must go straight to a room

Thoughts:
- I'll call amphipods in the hallway "Parked" and those in the rooms "Housed"
- Single object representing the burrow that can track all possible moves?
- State is parametrized by the positions of the 8 amphipods (16 params), the score, their state, and the one that moved last
    - Can I cache it?
    - 19 possible positions for them, so 19x18x17x16x15x14x13x12 = 3,047,466,240 = 3B states (excluding score) so might run out of RAM if we need to explore too much of the solution space
    - Define amphipods_state = 0 for hasn't moved, 1 for has moved into hallway, 2 for has moved into room
- Maybe Dijkstra's algorithm?
- What are the actual choices for a given move?
    - Park one amphipod out of 4. 11 possible positions, 4 possible amphipods
    - Park or House another amphipod. Max 11 possible positions and 7 amphipods to move
    - More generally, each turn there are 7 amphipods to move (can't move last amphipod) and 11 positions they can move to. So 77 choices. But most will be invalid

In [1]:
from collections import defaultdict
from functools import lru_cache
import time
from queue import PriorityQueue
class Burrow(object):
    def __init__(self,fname, part2 = False):
        """Read in a file with a new burrow object"""
        with open(fname, "r") as f:
            data = f.read().splitlines()

        self.part2 = part2
        
        if part2:
            self.n_group = 4 # 4 of each type
            data.insert(3,"  #D#C#B#A#  ")
            data.insert(4,"  #D#B#A#C#  ")
        else:
            self.n_group = 2 # 2 of each type
            
        self.ny = len(data)
        self.nx = len(data[0])
        self.best_cost = 9999999

        self.map = data
            
        # Clean up the map for printing
        self.clean_map = [l.replace("A",".").replace("B",".").replace("C",".").replace("D",".") for l in data]

        # Save the position of each amphipod
        pos = []
        for y in range(self.ny):
            for x in range(self.nx):
                char = data[y][x]
                if char in "ABCD":
                    pos.append((char,x,y))
                    
        # Turn it into a state array
        # Each entry is [state,x,y]
        # So state[0:2] is for A1 and A2 (for part 1)
        # state[2:4] is for B1 and B2 etc.
        amp_state = []
        for l in 'ABCD':
            for a in pos:
                if a[0] == l:
                    amp_state.append((0,a[1],a[2]))
        
        self.initial_amp_state = tuple(amp_state)
        
        # Save a list of all spaces
        all_spaces = set()
        for y in range(self.ny):
            for x in range(self.nx):
                if data[y][x] in ".ABCD":
                    all_spaces.add((x,y))
        # Can't stop in an open doorway
        all_spaces.discard((3,1))
        all_spaces.discard((5,1))
        all_spaces.discard((7,1))
        all_spaces.discard((9,1))
        self.all_spaces = all_spaces
        
    def find_available_spaces(self, amp_state):
        """Return a list of empty spaces that we could move an amphipod to."""
        open_spaces = self.all_spaces.copy()
        for _,x,y in amp_state:
            open_spaces.discard((x,y))
                    
        return open_spaces
    
    @lru_cache(maxsize=None)
    def get_path(self,pos1,pos2):
        """Get the list of positions needed to move from pos1 to pos2.
        Will return a set, so not in order. Can cache this since it will
        be reused a lot.
        """
        x1,y1 = pos1
        x2,y2 = pos2

        path = set()
        # Are they in a vertical line?
        if x1 == x2:
            path = set([(x1,y) for y in range(min(y1,y2),max(y1,y2)+1)])
            # Remove starting point
            path.remove(pos1)

        else:
            # Otherwise we can follow the same U shaped path through the hallway
            # Move until we're in the hallway
            for y in range(y1,1,-1):
                path.add((x1,y))
            # Move across until we're in line
            for x in range(min(x1,x2),max(x1,x2)+1):
                path.add((x,1))
            # Move vertically until we're there
            for y in range(1,y2+1):
                path.add((x2,y))
        if pos1 in path:
            path.remove(pos1)
        return path
    
    def check_path(self,path,amp_state):
        """Check if a path can be taken without moving through another amphipod."""
        for _,x,y in amp_state:
            pos = (x,y)
            if pos in path:
                return False
        return True
    
    def would_block_another_amphipod(self,amp_state,x,amp_type):
        """Check if moving an amphipod of type amp_type into pos
        would block an amphipod into the wrong house.
        """
        # This should be equivalent to checking if there are other types of amphipods with the same x
        for ix,amp in enumerate(amp_state):
            if ix//self.n_group != amp_type:
                if amp[1] == x:
                    return True
        return False
    
    def best_possible_cost_from_here(self,amp_state):
        """What is the best possible cost to get the right solution from here,
        if we allow amphipods to move through each other?
        """
        cost = 0
        for ix,amp in enumerate(amp_state):
            state,x,y = amp
            amp_type = ix//self.n_group
            movement_cost = 10**(ix//self.n_group)
            final_y = ((ix%self.n_group)+2) # This should work in almost all cases
            final_x = (amp_type*2+3)

            if state == 2:
                moves = 0
            elif (x==final_x) and (y>final_y):
                # If this is already too far down, we gain some moves from counting later
                moves = (final_y-y)
            elif x == final_x:
                moves = abs(y-final_y)
            else:
                # Need to move up and then down
                moves = (y-1) + abs(x-final_x) + (final_y-1)
            cost += moves*movement_cost
        return cost
    
    def get_available_moves(self,amp_state,cost,last_amphipod_moved):
        """Find all available moves from a given amp state."""
        moves = []
        # Which spaces are available
        open_spaces = self.find_available_spaces(amp_state)
        
        # Loop through amphipods we can move
        for ix,this_amp in enumerate(amp_state):
            # Don't move the same amphipod twice
            if ix == last_amphipod_moved:
                continue
                
            state,old_x,old_y = this_amp
            this_amp_type = ix//self.n_group # A=0,B=1,C=2,D=3
            this_amp_movement_cost = 10**(ix//self.n_group)
                
            # If we're already housed, don't move this amphipod
            if state ==2:
                continue
                            
            # Check if we can move to each open space
            for x,y in open_spaces:
                new_amp_state = list(amp_state[:]).copy()
                invalid = False
                
                # First, some invalid moves
                if y==1 and state ==1:
                    # This would be a hallway-hallway move so we cant do it
                    continue
                elif (y>=2 and y<=self.n_group) and (x,self.n_group+1) in open_spaces:
                    # Don't move into the top of an empty room
                    continue
                elif y >1 and x != (this_amp_type*2+3):
                    # Only move into your correct room
                    continue
                elif ((old_y-y)>0) and (x==old_x):
                    ### Don't move up unless you're leaving the room
                    continue
                elif (y>=2 and y<=self.n_group):
                    ### Don't block another amphipod into the wrong room
                    if self.would_block_another_amphipod(amp_state,x,this_amp_type):
                        continue
                
                # Now check whether we can actually move there
                path = self.get_path((old_x,old_y),(x,y))
                path_ok = self.check_path(path,amp_state)
                
                if path_ok:
                    # Make a new state and calculate new cost
                    if y==1:
                        new_state = 1
                    else:
                        new_state = 2
                    new_amp_state[ix] = (new_state,x,y)
                    extra_cost = len(path)*this_amp_movement_cost

                    moves.append((tuple(new_amp_state),cost+extra_cost,ix))
        return moves 
    
    def check_completed(self,amp_state):
        """Check if an amp state is a completed state."""
        for ix,amp in enumerate(amp_state):
            _,x,y = amp
            amp_type = ix//self.n_group
            if x != (amp_type*2+3):
                return False
        return True
            
    
    def solve_from_here(self,amp_state,cost,last_amphipod_moved):
        """Work out the best way to solve the problem from here."""
        
        # Loop through all possible options
        options_to_check = defaultdict(set)
        best_possible_cost = self.best_possible_cost_from_here(amp_state)
        options_to_check[best_possible_cost].add((amp_state,cost,last_amphipod_moved))
        
        states_checked = set()
        states_in_queue = set()
        
        counter = 0
        checking_cost = best_possible_cost
        
        while True:
            
            # If an entry with this cost exists, then explore moves from there
            if len(options_to_check[checking_cost]) > 0:
                amp_state,cost,last_amphipod_moved = options_to_check[checking_cost].pop()
            else:
                ### Otherwise remove it from the queue and try again with the smallest cost
                options_to_check.pop(checking_cost)
                checking_cost = min(options_to_check.keys())
                # Stop if we're checking entries that are worse than the best solution we have
                if checking_cost > self.best_cost:
                    return self.best_cost
                continue
                
            if cost > self.best_cost:
                continue
                
            # Don't check a less-efficient way to get to the same spot
            if amp_state in states_checked:
                continue
            
            counter +=1
            
            # Check if this solution is solved
            is_completed = self.check_completed(amp_state)
            if is_completed:
                print("Solution:",cost)
                self.best_cost = min(cost,self.best_cost)
            
            # Otherwise, find some valid moves and add them to the queue
            valid_moves = self.get_available_moves(amp_state,cost,last_amphipod_moved)
            for v in valid_moves:
                if ((v[0],v[1])) in states_in_queue:
                    continue
                best_cost = v[1]+self.best_possible_cost_from_here(v[0])
                options_to_check[best_cost].add(v)
                states_in_queue.add((amp_state,cost))
            
            states_checked.add(amp_state)
            if (counter % 25000) == 0:
                print("Checked",counter,"Up to cost",checking_cost)
            
        return self.best_cost
            
        
    def print_burrow(self,amp_state):
        """Print the burrow for a given amp state, for debugging."""
        current_map = self.clean_map.copy()
        for ix,amp in enumerate(amp_state):
            _,x,y = amp
            row = current_map[y]
            row = [a for a in row]
            row[x] = ["A","B","C","D"][ix//self.n_group]
            current_map[y] = "".join(row)
            
        for l in current_map:
            print(l)

In [2]:
%%time
fname = "inputs/day23_test_input.dat"
burrow = Burrow(fname)

amp_state = burrow.initial_amp_state
# amp_state = ((2,3,3),(2,3,2),(2,5,2),(2,5,3),(2,7,3),(1,4,1),(2,9,3),(0,1,1))
burrow.print_burrow(amp_state)
burrow.solve_from_here(amp_state,0,-1) # 12521

#############
#...........#
###B#C#B#D###
  #A#D#C#A#  
  #########  
Solution: 12521
CPU times: user 1.63 s, sys: 14.9 ms, total: 1.65 s
Wall time: 1.65 s


12521

In [3]:
%%time
# Puzzle input
fname = "inputs/day23_input.dat"
burrow = Burrow(fname)

amp_state = burrow.initial_amp_state
burrow.solve_from_here(amp_state,0,-1) # 14627

Solution: 14629
CPU times: user 295 ms, sys: 2.24 ms, total: 297 ms
Wall time: 296 ms


14629

### Part 2:
- The burrows are 2 squares deeper and there are 4 amphipods
- Need to insert in between the top and bottom of the houses
    -  #D#C#B#A#
    -  #D#B#A#C#
- Approach:
    - I generalized the code above to 4 amphipods
    - Also moved from using a Dijkstra-like approach to something closer to A*, where I estimate the minimum total movement cost from a given state and do a search with a queue starting from the best minimum possible value.
    - Also swapped the queue to a dictionary of sets, so I can't put the same state in twice, although this could be done better by 

In [4]:
%%time
fname = "inputs/day23_test_input.dat"
burrow = Burrow(fname,part2=True)

amp_state = burrow.initial_amp_state
burrow.solve_from_here(amp_state,0,-1) #

Checked 25000 Up to cost 36487
Checked 50000 Up to cost 36943
Checked 75000 Up to cost 37385
Checked 100000 Up to cost 37503
Checked 125000 Up to cost 37557
Checked 150000 Up to cost 37669
Checked 175000 Up to cost 37623
Checked 200000 Up to cost 38043
Checked 225000 Up to cost 38347
Checked 250000 Up to cost 38483
Checked 275000 Up to cost 39073
Checked 300000 Up to cost 39427
Checked 325000 Up to cost 39501
Checked 350000 Up to cost 39581
Checked 375000 Up to cost 39721
Checked 400000 Up to cost 39861
Checked 425000 Up to cost 40203
Checked 450000 Up to cost 40419
Checked 475000 Up to cost 40601
Checked 500000 Up to cost 40921
Checked 525000 Up to cost 41149
Checked 550000 Up to cost 41429
Checked 575000 Up to cost 41521
Checked 600000 Up to cost 41609
Checked 625000 Up to cost 41711
Checked 650000 Up to cost 41845
Checked 675000 Up to cost 42205
Checked 700000 Up to cost 42385
Checked 725000 Up to cost 42543
Checked 750000 Up to cost 42781
Checked 775000 Up to cost 43091
Checked 800

44169

In [5]:
%%time
fname = "inputs/day23_input.dat"
burrow = Burrow(fname,part2=True)

amp_state = burrow.initial_amp_state
burrow.solve_from_here(amp_state,0,-1) #

Checked 25000 Up to cost 41531
Checked 50000 Up to cost 42921
Checked 75000 Up to cost 43499
Solution: 41591
CPU times: user 13.8 s, sys: 52.7 ms, total: 13.9 s
Wall time: 13.9 s


41591